<p align="center">
Universidade Federal de Alagoas</br>

Instituto de Computação</br>

</p>

#Trabalho Prático AB2

Disciplina: Sistemas Distribuídos

Semestre letivo: 2016.1

Professor: André Lage Freitas

## Especificação

**Importante**. Leia atentamente essa seção pois só serão aceitos trabalhos que obedecerem as a especificações a seguir. 

Baseado(a) no exemplo a seguir de implementação do B-multicast em Julia, façam os exercícios descritos nesse documento.

In [2]:
function B_multicast(g,m,B_deliver)
    for p in g
        remotecall(p, B_deliver, m)
    end
end
 
# Adding 10 processes (workers)
rmprocs(procs())
addprocs(10)

# My group is composed by all workers
g = workers()

times = Any[]
# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
for i in 1:51
    tic()
    B_multicast(g,"oi!",println)
    push!(times,toc())
end

sleep(10)
println(times)

elapsed time: 0.003788428 seconds
elapsed time: 0.000288813 seconds
elapsed time: 0.000290131 seconds
elapsed time: 0.000281562 seconds
elapsed time: 0.00027419 seconds
elapsed time: 0.000268354 seconds
elapsed time: 0.027528162 seconds
elapsed time: 0.000273848 seconds
elapsed time: 0.000274937 seconds
elapsed time: 0.000268006 seconds
elapsed time: 0.000320729 seconds
elapsed time: 0.000275754 seconds
elapsed time: 0.000260816 seconds
elapsed time: 0.000263891 seconds
elapsed time: 0.00027318 seconds
elapsed time: 0.000251168 seconds
elapsed time: 0.000249293 seconds
elapsed time: 0.000243892 seconds
elapsed time: 0.000250783 seconds
elapsed time: 0.000229863 seconds
elapsed time: 0.000230937 seconds
elapsed time: 0.00023719 seconds
elapsed time: 0.000309475 seconds
elapsed time: 0.000233451 seconds
elapsed time: 0.000235148 seconds
elapsed time: 0.000181966 seconds
elapsed time: 0.023399005 seconds
elapsed time: 0.000200718 seconds
elapsed time: 0.000195641 seconds
elapsed time: 0.0

### Exercício 1: Sincronismo (2,0 pontos)

Adicione 10 processos com o comando `addprocs` abaixo. 

_Atenção_: sua instalação local de Julia pode não conseguir adicionar 10 processos, use a [JuliaBox](https://juliabox.org).

In [3]:
#rmprocs(procs())
#addprocs(10)

10-element Array{Any,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

Modifique a implementação do B-multicast para utilizar communicação síncrona.

In [3]:
function sync_B_multicast(g,m,B_deliver)
    for p in g
        remotecall_wait(p, B_deliver, m)
    end
end

# My group is composed by all workers
g = workers()

# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
times = Any[]
# Multicasting "oi!" message to group `g` by assuming `println` as B_deliver function.
for i in 1:51
    tic()
    B_multicast(g,"oi!",println)
    push!(times,toc())
end
println(times)

	From worker 12:	oi!
	From worker 13:	oi!
	From worker 14:	oi!
	From worker 15:	oi!
	From worker 16:	oi!
	From worker 17:	oi!
	From worker 18:	oi!
elapsed time: 	From worker 19:	oi!
0.001548739 seconds
	From worker 20:	oi!
	From worker 21:	oi!
	From worker 12:	oi!
	From worker 13:	oi!
	From worker 14:	oi!
	From worker 15:	oi!
	From worker 16:	oi!
	From worker 17:	oi!
	From worker 18:	oi!
	From worker 19:	oi!
elapsed time: 0.0	From worker 20:	oi!
0	From worker 21:	oi!
0703784 seconds
	From worker 12:	oi!
	From worker 13:	oi!
	From worker 14:	oi!
	From worker 16:	oi!
	From worker 15:	oi!
	From worker 17:	oi!
	From worker 18:	oi!
	From worker 19:	oi!
elapsed time: 0.	From worker 20:	oi!
0	From worker 21:	oi!
00636095 seconds
	From worker 12:	oi!
	From worker 13:	oi!
	From worker 14:	oi!
	From worker 16:	oi!
	From worker 15:	oi!
	From worker 17:	oi!
	From worker 18:	oi!
	From worker 19:	oi!
elapsed time: 	From worker 20:	oi!
0.	From worker 21:	oi!
000616048 seconds
	From worker 12:	oi!
	Fr

Execute as duas versões do B-multicast (síncrona e assíncrona) e compare as saídas.

A ordem das mensagens das saídas foram as mesmas para ambas versões do B-multicast? Por quê?

**Resposta**: _Não, pois ao tornar o B_multicast síncrono, a cada chamada tivemos que aguardar sua resposta, então a saída ficou ordenada de acordo com as chamadas dos processos. No caso do B_multicast assíncrono observamos que não há uma ordem na saída dos processos, aquele que for encerrando suas atividades primeiro vai imprimindo a saída. _



### Exercício 2: Análise de desempenho do B-multicast (2,0 pontos)

Meça o tempo de execução duas versões do B-multicast (síncrona e assíncrona). Dica: utilize a macro `@time`.

* Execute 50 vezes essa medição e construa um gráfico em forma de barras:
	* cada barra representando cada versão do B-multicast;
	* calcule e coloque no gráfico o [intervalo de confiança](https://en.wikipedia.org/wiki/Confidence_interval) com nível de 95%.
    
**Gráfico**: <img src="images/B-multicast.png">
    
    
* Qual das duas versões teve o menor tempo de execução? Por quê?

**Resposta**: _A versão síncrona, por ser de simples execução, em um pequeno número de repetições, o algoritmo síncrono apresenta melhor desempenho. 
O algoritmo assíncrono necessita se comunicar por flags, avisando quando o resultado está disponível, tornando mais lento neste caso. Apesar de apresentar o melhor resultado 
neste teste, o B-multicast síncrono não é escalável, pois a quantidade de processos envolvidos afetaram o desempenho diretamente._



### Exercício 3: Análise de desempenho do R-multicast (3,0 pontos)

Implemente duas versões do algoritmo R-multicast: uma assíncrona e outra síncrona.

* R-multicast síncrono

In [13]:
@everywhere messages = Any[]
@everywhere has_received = Any[]

g = workers()

function sync_B_multicast(p,g,m,B_deliver)
    # checking message integrity
    if m in messages
        # checking if the process already got the message.
        if !(p in has_received)
            push!(has_received, p) # process confirms receiving.
            remotecall_wait(p, B_deliver, m) # call the execution
        end
    end
end

function sync_R_multicast(g,m,R_deliver)
    # starting multicast
    push!(messages,m) # adding the actual message to an array. 
    for p in g
        sync_B_multicast(p,g,m,R_deliver)
    end
    
    # if some process failed to receive the message, try it again.
    if !(has_received == g)
        sync_R_multicast(g,m,R_deliver)
    end
end

# Multicasting "oi!" message to group `g`
times = Any[]
for i in 1:51
    tic()
        sync_R_multicast(g, "oi!", println)
    push!(times,toc())
end

println(times)

	From worker 2:	oi!
	From worker 3:	oi!
	From worker 4:	oi!
	From worker 5:	oi!
	From worker 6:	oi!
	From worker 7:	oi!
	From worker 8:	oi!
	From worker 9:	oi!
	From worker 10:	oi!
	From worker 11:	oi!
elapsed time: 0.013142072 seconds
elapsed time: 1.9941e-5 seconds
elapsed time: 7.695e-6 seconds
elapsed time: 7.73e-6 seconds
elapsed time: 7.905e-6 seconds
elapsed time: 7.992e-6 seconds
elapsed time: 8.02e-6 seconds
elapsed time: 8.333e-6 seconds
elapsed time: 9.393e-6 seconds
elapsed time: 8.907e-6 seconds
elapsed time: 9.14e-6 seconds
elapsed time: 9.24e-6 seconds
elapsed time: 9.588e-6 seconds
elapsed time: 9.955e-6 seconds
elapsed time: 1.0018e-5 seconds
elapsed time: 1.0698e-5 seconds
elapsed time: 1.1165e-5 seconds
elapsed time: 1.1116e-5 seconds
elapsed time: 1.1697e-5 seconds
elapsed time: 1.1499e-5 seconds
elapsed time: 1.1807e-5 seconds
elapsed time: 1.2173e-5 seconds
elapsed time: 1.2262e-5 seconds
elapsed time: 1.2582e-5 seconds
elapsed time: 1.2654e-5 seconds
elapsed time

* R-multicast assíncrono

In [17]:
@everywhere messages = Any[]
@everywhere has_received = Any[]

g = workers()

function B_multicast(p,g,m,B_deliver)
    # checking message integrity
    if m in messages
        # checking if the process already got the message.
        if !(p in has_received)
            push!(has_received, p) # process confirms receiving.
            remotecall(p, B_deliver, m) # call the execution
        end
    end
end

function R_multicast(g,m,R_deliver)
    # starting multicast
    push!(messages,m) # adding the actual message to an array. 
    for p in g
        B_multicast(p,g,m,R_deliver)
    end
    
    # if some process failed to receive the message, try it again.
    if !(has_received == g)
        R_multicast(g,m,R_deliver)
    end
end

# Multicasting "oi!" message to group `g`
times = Any[]
for i in 1:51
    tic()
        R_multicast(g, "oi!", println)
    push!(times,toc())
end

println(times)

	From worker 12:	oi!
	From worker 13:	oi!
	From worker 14:	oi!
	From worker 15:	oi!
	From worker 16:	oi!
	From worker 17:	oi!
	From worker 18:	oi!
	From worker 19:	oi!
elapsed time: 	From worker 20:	oi!
0.007015128 seconds	From worker 21:	oi!

elapsed time: 8.623e-6 seconds
elapsed time: 7.628e-6 seconds
elapsed time: 8.26e-6 seconds
elapsed time: 2.5332e-5 seconds
elapsed time: 8.05e-6 seconds
elapsed time: 7.837e-6 seconds
elapsed time: 8.147e-6 seconds
elapsed time: 8.639e-6 seconds
elapsed time: 8.84e-6 seconds
elapsed time: 8.912e-6 seconds
elapsed time: 9.009e-6 seconds
elapsed time: 9.203e-6 seconds
elapsed time: 9.789e-6 seconds
elapsed time: 9.703e-6 seconds
elapsed time: 1.0514e-5 seconds
elapsed time: 1.1092e-5 seconds
elapsed time: 1.0908e-5 seconds
elapsed time: 1.0857e-5 seconds
elapsed time: 1.1598e-5 seconds
elapsed time: 1.1575e-5 seconds
elapsed time: 1.1738e-5 seconds
elapsed time: 1.2252e-5 seconds
elapsed time: 1.2267e-5 seconds
elapsed time: 1.2221e-5 seconds
elap

Faça a mesma análise de desempenho para o R-multicast que você fez no Exercício 2.


**Gráfico**: <img src="images/R-multicast.png">   
    
    
* Qual das duas versões (R-multicast assíncrono e assíncrono) teve o menor tempo de execução? Por quê?

**Resposta**: _O R-multicast assíncrono apresenta um melhor resultado. Diferente do B-multicast, onde há apenas o envio da mensagem sem nenhuma checagem, 
no R-multicast nós checamos a integridade da mensagem, e verificamos se todos os processos já receberam. Caso um processo já tenha recebido a mensagem, 
não há necessidade de repetição, não como o B-multicast, onde independente se o processo recebeu ou não a mensagem, ele irá realizar a chamada de procedimento remoto novamente.  
Assim, percebemos que o R-multicast assíncrono terá um tempo menor de execução pois não precisa aguardar um outro processo finalizar a checagem para retornar o resultado._



### Exercício 4: Análise de desempenho: B-multicast vs. R-multicast (3,0 pontos)

Para cada uma das comparações abaixo, qual dos algoritmos teve o menor tempo de execução? Por quê?

* B-multicast assíncrono vs. R-multicast assíncrono.
    

**Resposta**: _O R-multicast assíncrono apresenta menor tempo de execução. Apesar de apresentar checagem de mensagem, o R-multicast não irá retransmitir caso o processo já tenha recebido a mensagem, diferente do B-multicast, assim tornando mais eficiente._    
    
* B-multicast síncrono vs. R-multicast síncrono.


**Resposta**: _O R-multicast síncrono possui menor tempo de execução. Por razões semelhantes ao que já foi discutido anteriormente, checagem de mensagem e retransmissão._

## Entrega

As respostas deverão ser entregues nos devidos campos desse arquivo, ou seja, em [Markdown](https://en.wikipedia.org/wiki/Markdown) e no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl).

O programa deve ser implementado na linguagem de programação **Julia**. Seu trabalho será testado na [JuliaBox](https://juliabox.org), portanto teste seu arquivo IJulia na JulizBox antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente por e-mail** para o seguinte endereço: **prof.alage** (no) **gmail com**


Será registrada como data de recebimento do trabalho a data indicada pelo servidor de e-mail do Professor. 

A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos ou se o e-mail não chegar à caixa de entrada de e-mail do Professor.

### Prazos

O prazo de entrega sem atraso é o dia **12/10/16**.

Trabalhos entregues com atraso serão aceitos, no máximo, até o o dia **16/10/16**. Serão descontados 5% da nota obtida por dia de atraso.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.